In [1]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
import numpy as np

# Load the diabetes dataset
diabetes = load_diabetes()

# Generate train-test splits
X_train, X_test, y_train, y_test = train_test_split(diabetes.data[:, np.newaxis, 2], diabetes.target, test_size=0.33, random_state=42)

In [2]:
from flex.data import FedDataDistribution, Dataset

train_diabetes = Dataset.from_numpy(X_train, y_train)
test_diabetes = Dataset.from_numpy(X_test, y_test)

federated_diabetes = FedDataDistribution.iid_distribution(train_diabetes, n_clients=5)

In [3]:
from sklearn.linear_model import LinearRegression
from flex.pool import init_server_model
from flexBlock.pool import DecoupledBlockchainPool
from flex.model import FlexModel


@init_server_model
def build_server_model():
    flex_model = FlexModel()
    flex_model["model"] = LinearRegression()
    return flex_model


p = DecoupledBlockchainPool(federated_diabetes, number_of_miners=2, init_func=build_server_model)

servers = p.servers
aggregators = p.aggregators
clients = p.clients

print(f"Number of nodes in the pool {len(p.actor_ids)}: {len(servers)} servers plus {len(clients)} clients. The server is also an aggregator")

Number of nodes in the pool 7: 2 servers plus 5 clients. The server is also an aggregator


In [4]:
import copy

from flex.pool import deploy_server_model

@deploy_server_model
def copy_server_model_to_clients(server_flex_model: FlexModel):
    return copy.deepcopy(server_flex_model)

servers.map(copy_server_model_to_clients, clients)

In [5]:
def train(client_flex_model: FlexModel, client_data: Dataset):
    client_flex_model["model"].fit(client_data.X_data, client_data.y_data)

clients.map(train)

In [6]:
from flexBlock.pool import send_weights_to_miner

@send_weights_to_miner
def get_clients_weights(client_flex_model: FlexModel):
    return [client_flex_model["model"].intercept_, client_flex_model["model"].coef_]

aggregators.map(get_clients_weights, clients)

# Aqui tengo que hacer que los mineros verifiquen que los pesos que les llegan son correctos y filtrar

In [7]:
from flexBlock.pool import validate_models

@validate_models
def validate_all_models(model):
    return True

aggregators.map(validate_all_models, clients)

In [8]:
aggregators._models

{'server-1': {'clients_connections': array([4, 0, 2]), 'model': LinearRegression(), 'weights': [[149.62065538236186, array([1052.32054866])], [155.03030613967434, array([938.74359785])], [148.35799346367716, array([956.94484324])]], 'valid_models': []},
 'server-2': {'clients_connections': array([1, 3]), 'model': LinearRegression(), 'weights': [[150.51018565933487, array([975.31732556])], [149.83316633729274, array([951.49409266])]], 'valid_models': []}}

In [9]:
p.gossip()
aggregators._models

{'server-1': {'clients_connections': array([4, 0, 2]), 'model': LinearRegression(), 'weights': [[149.62065538236186, array([1052.32054866])], [155.03030613967434, array([938.74359785])], [148.35799346367716, array([956.94484324])]], 'valid_models': []}, 'server-2': {'clients_connections': array([1, 3]), 'model': LinearRegression(), 'weights': [[150.51018565933487, array([975.31732556])], [149.83316633729274, array([951.49409266])]], 'valid_models': []}}


{'server-1': {'clients_connections': array([4, 0, 2]), 'model': LinearRegression(), 'weights': [[149.62065538236186, array([1052.32054866])], [155.03030613967434, array([938.74359785])], [148.35799346367716, array([956.94484324])], [150.51018565933487, array([975.31732556])], [149.83316633729274, array([951.49409266])]], 'valid_models': []},
 'server-2': {'clients_connections': array([1, 3]), 'model': LinearRegression(), 'weights': [[149.62065538236186, array([1052.32054866])], [155.03030613967434, array([938.74359785])], [148.35799346367716, array([956.94484324])], [150.51018565933487, array([975.31732556])], [149.83316633729274, array([951.49409266])]], 'valid_models': []}}

In [10]:
from flexBlock.pool.primitives import proof_of_work
selected_miner = p._run_concensus(proof_of_work, [])
# Ahora agregar
selected_miner

'server-1'

In [11]:
from flex.pool import aggregate_weights
import numpy as np

@aggregate_weights
def aggregate(list_of_weights: list):
    return np.mean(np.asarray(list_of_weights, dtype=object), axis=0)

# Aggregate weights
aggregators.map(aggregate)

/home/mariogmarq/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/mariogmarq/.local/lib/python3.10/site-packages/numpy/core/_methods.py:131: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret / rcount


In [12]:
aggregators._models

{'server-1': {'clients_connections': array([4, 0, 2]), 'model': LinearRegression(), 'weights': [], 'valid_models': [], 'aggregated_weights': nan},
 'server-2': {'clients_connections': array([1, 3]), 'model': LinearRegression(), 'weights': [], 'valid_models': [], 'aggregated_weights': nan}}